# 1. Imports

In [ ]:
# Configs
%load_ext autoreload
%autoreload 3
## other standard packages
import sys
## Env variables and preparation stuffs
sys.path.insert(0, "../")
from src_code.data_utils.dataset import GridDataset
from src_code.data_utils.dataset_utils import CellType

# 2. Dataset

In [ ]:
dataset = GridDataset(grid_size=5, seed = 42, wall_symbol="#", free_symbol=".")

img_rgb1, grid_world1 = dataset[1]

In [ ]:
img_rgb1

In [ ]:
print(str(grid_world1))

In [ ]:
print(f"{grid_world1.a_star() = }")

## 3 Open Flamingo Inference
Source: https://github.com/mlfoundations/open_flamingo

## 3.1  Reference

In [ ]:
from open_flamingo import create_model_and_transforms

model, image_processor, tokenizer = create_model_and_transforms(
    clip_vision_encoder_path="ViT-L-14",
    clip_vision_encoder_pretrained="openai",
    lang_encoder_path="togethercomputer/RedPajama-INCITE-Instruct-3B-v1",
    tokenizer_path="togethercomputer/RedPajama-INCITE-Instruct-3B-v1",
    cross_attn_every_n_layers=2
)

# grab model checkpoint from huggingface hub
from huggingface_hub import hf_hub_download
import torch

checkpoint_path = hf_hub_download("openflamingo/OpenFlamingo-4B-vitl-rpj3b-langinstruct", "checkpoint.pt")
model.load_state_dict(torch.load(checkpoint_path), strict=False)


In [ ]:
# grab model checkpoint from huggingface hub
from huggingface_hub import hf_hub_download
import torch
from PIL import Image
import requests
device = "cpu"
# if torch.cuda.is_available():
#     print("CUDA is available! Using GPU for calculations.")
#     device = torch.device("cuda")
# else:
#     print("CUDA is not available. Using CPU for calculations.")
#     device = torch.device("cpu")
model = model.to(device)

In [ ]:
from PIL import Image
import requests
import torch

"""
Step 1: Load images
"""
demo_image_one = Image.open(
    requests.get(
        "http://images.cocodataset.org/val2017/000000039769.jpg", stream=True
    ).raw
)

demo_image_two = Image.open(
    requests.get(
        "http://images.cocodataset.org/test-stuff2017/000000028137.jpg",
        stream=True
    ).raw
)

query_image = Image.open(
    requests.get(
        "http://images.cocodataset.org/test-stuff2017/000000028352.jpg", 
        stream=True
    ).raw
)


"""
Step 2: Preprocessing images
Details: For OpenFlamingo, we expect the image to be a torch tensor of shape 
 batch_size x num_media x num_frames x channels x height x width. 
 In this case batch_size = 1, num_media = 3, num_frames = 1,
 channels = 3, height = 224, width = 224.
"""
vision_x = [image_processor(demo_image_one).unsqueeze(0), image_processor(demo_image_two).unsqueeze(0), image_processor(query_image).unsqueeze(0)]
vision_x = torch.cat(vision_x, dim=0)
vision_x = vision_x.unsqueeze(1).unsqueeze(0).to(device)

"""
Step 3: Preprocessing text
Details: In the text we expect an <image> special token to indicate where an image is.
 We also expect an <|endofchunk|> special token to indicate the end of the text 
 portion associated with an image.
"""
tokenizer.padding_side = "left" # For generation padding tokens should be on the left
lang_x = tokenizer(
    ["<image>An image of two cats.<|endofchunk|><image>An image of a bathroom sink.<|endofchunk|><image>An image of"],
    return_tensors="pt",
)


"""
Step 4: Generate text
"""
generated_text = model.generate(
    vision_x=vision_x,
    lang_x=lang_x["input_ids"],
    attention_mask=lang_x["attention_mask"],
    max_new_tokens=20,
    num_beams=3,
)

print("Generated text: ", tokenizer.decode(generated_text[0]))

## 3.2 Few Shot Learning - only rgb input

In [ ]:
from PIL import Image
import requests
import torch
import numpy as np
"""
Step 1: Load images
"""
dataset = GridDataset(grid_size=5, seed = 42, wall_symbol="#", free_symbol=".", cell_size=5)
img_rgb1, gridworld1 = dataset[0]
img_rgb2, gridworld2 = dataset[1]
img_rgb3, gridworld3 = dataset[2]
img_rgb4, gridworld4 = dataset[3]
ascii_inp1, path1 = str(gridworld1), gridworld1.a_star()
ascii_inp2, path2 = str(gridworld2), gridworld2.a_star()
ascii_inp3, path3 = str(gridworld3), gridworld3.a_star()
ascii_inp4, path4 = str(gridworld4), gridworld4.a_star()

In [ ]:
from src_code.data_utils.dataset_utils import draw_image_grid
draw_image_grid([(img_rgb1, path1), (img_rgb2, path2), (img_rgb3, path3)])

## OpenFlamingo RGB Prompt

In [ ]:
from src_code.data_utils.prompt_utils import generate_inputs_for_openflamingo_rgb_and_text
dataset = GridDataset(grid_size=5, seed = 42, wall_symbol="#", free_symbol=".")
query_idx=1
vision_x, lang_x, str_prompt = generate_inputs_for_openflamingo_rgb_and_text(tokenizer, image_processor, dataset, query_idx=query_idx, img_symbol="<image>")
print(str_prompt)

In [ ]:
vision_x[0][0].shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(vision_x[0][1][0].permute(1, 2, 0))

In [ ]:
dataset[1][0]

### 3.2.1 One Shot Learning RGB and Text

In [ ]:
for query_idx in range(10):

    vision_x, lang_x, str_prompt = generate_inputs_for_openflamingo_rgb_and_text(tokenizer, image_processor, dataset, img_symbol="<image>", query_idx=query_idx)
    
    """
    Step 4: Generate text
    """
    generated_text = model.generate(
        vision_x=vision_x,
        lang_x=lang_x["input_ids"],
        attention_mask=lang_x["attention_mask"],
        max_new_tokens=20,
        num_beams=3,
    )
    
    print("Generated text: ", tokenizer.decode(generated_text[0]))

### 3.2.2 only Text

#### OpenFlamingo Only Text Prompt

In [ ]:
from src_code.data_utils.prompt_utils import generate_inputs_for_openflamingo_text
dummy_vision_x, lang_x, str_prompt = generate_inputs_for_openflamingo_text(tokenizer, image_processor, dataset, query_idx=1, pure_language=True)
print(str_prompt)

In [ ]:
for query_idx in range(1):
    dummy_vision_x, lang_x, str_prompt = generate_inputs_for_openflamingo_text(tokenizer, image_processor, dataset, query_idx=query_idx, pure_language=True)
    """
    Step 4: Generate text
    """
    generated_text = model.generate(
        vision_x=dummy_vision_x,
        lang_x=lang_x["input_ids"],
        attention_mask=lang_x["attention_mask"],
        max_new_tokens=20,
        num_beams=3,
    )
    
    print("Generated text: ", tokenizer.decode(generated_text[0]))

### 3.2.3  only ASCII Input

#### OpenFlamingo Only ASCII

In [ ]:
from src_code.data_utils.prompt_utils import generate_inputs_for_openflamingo_text
dummy_vision_x, lang_x, str_prompt = generate_inputs_for_openflamingo_text(tokenizer, image_processor, dataset, query_idx=1, pure_language=False)
print(str_prompt)

In [ ]:
from src_code.data_utils.prompt_utils import generate_inputs_for_openflamingo_text

"""
Step 4: Generate text
"""
for query_idx in range(1):
    dummy_vision_x, lang_x, str_prompt = generate_inputs_for_openflamingo_text(tokenizer, image_processor, dataset, query_idx=query_idx, pure_language=False)
    generated_text = model.generate(
        vision_x=dummy_vision_x,
        lang_x=lang_x["input_ids"],
        attention_mask=lang_x["attention_mask"],
        max_new_tokens=20,
        num_beams=3,
    )
    print("Generated text: \n", tokenizer.decode(generated_text[0]))

# What we tried.
1. directly prompt - garbage answer
2. one shot propmt - ???
3. two shot prompt - doesnt work
4. different image resolutions
5. ascii
6. bigger models??